In [70]:
#This module aims at forecasting stock prices by 'n' days into the future using machine learning

In [71]:
#Import quadl to retrieve historical stock data

In [72]:
pip install quandl

Note: you may need to restart the kernel to use updated packages.


In [94]:
#import the dependencies 
import quandl
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split  #this is necessary to split and train our data
stock = input('Insert Ticker HERE : ')

Insert Ticker HERE :  AAPL


In [95]:
#Get the stock data identiied as the variable 'df' standing for 'dataframe'

df = quandl.get('WIKI/'+ stock)
#Take a look at the headers of your data to understand which are useful
print(df.head())


             Open   High    Low  Close     Volume  Ex-Dividend  Split Ratio  \
Date                                                                          
1980-12-12  28.75  28.87  28.75  28.75  2093900.0          0.0          1.0   
1980-12-15  27.38  27.38  27.25  27.25   785200.0          0.0          1.0   
1980-12-16  25.37  25.37  25.25  25.25   472000.0          0.0          1.0   
1980-12-17  25.87  26.00  25.87  25.87   385900.0          0.0          1.0   
1980-12-18  26.63  26.75  26.63  26.63   327900.0          0.0          1.0   

            Adj. Open  Adj. High  Adj. Low  Adj. Close  Adj. Volume  
Date                                                                 
1980-12-12   0.422706   0.424470  0.422706    0.422706  117258400.0  
1980-12-15   0.402563   0.402563  0.400652    0.400652   43971200.0  
1980-12-16   0.373010   0.373010  0.371246    0.371246   26432000.0  
1980-12-17   0.380362   0.382273  0.380362    0.380362   21610400.0  
1980-12-18   0.391536   0.

In [96]:
#Use Adj.close Price
df = df[['Adj. Close']]
print(df.head())

            Adj. Close
Date                  
1980-12-12    0.422706
1980-12-15    0.400652
1980-12-16    0.371246
1980-12-17    0.380362
1980-12-18    0.391536


In [97]:
#Define the variable determining how many days into the future you want to forecast
#For the purpose of this module it will be set at 30 days standard
forecast_out = 30
#Create another column 'Target' shifted 'n' units up 
df['Prediction'] = df[['Adj. Close']].shift(- forecast_out)
#printing the head and tail of the dataset its possible to notice that the 'Adj. Close' column has been shifted by
# the 'n' units defined in 'forecast_out'
print(df.head())
print(df.tail())

            Adj. Close  Prediction
Date                              
1980-12-12    0.422706    0.470490
1980-12-15    0.400652    0.455787
1980-12-16    0.371246    0.439173
1980-12-17    0.380362    0.415355
1980-12-18    0.391536    0.391536
            Adj. Close  Prediction
Date                              
2018-03-21     171.270         NaN
2018-03-22     168.845         NaN
2018-03-23     164.940         NaN
2018-03-26     172.770         NaN
2018-03-27     168.340         NaN


In [98]:
#Create the independent dataset 'x' 
    # Convert the dataframe to a numpy array
X = np.array(df.drop(['Prediction'],1))
#Remove the last 'n' rows 
X = X[:-forecast_out]
print(X)

[[  0.42270592]
 [  0.40065169]
 [  0.37124607]
 ...
 [155.32      ]
 [155.97      ]
 [162.71      ]]


/var/folders/sr/18syqzcs2dzfcvt5l8sn7b680000gn/T/ipykernel_76913/1178484858.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  X = np.array(df.drop(['Prediction'],1))


In [99]:
#Create the dependent dataset 'y'
    #Convert the dataframe to a numpy array  (will give all values including the NaN)
y = np.array(df['Prediction'])
#get all of the y-values except the last 'n' rows
y = y[: -forecast_out]
print(y)

[  0.47049006   0.45578725   0.43917307 ... 164.94       172.77
 168.34      ]


In [100]:
### REMEMBER THAT X IS A LIST OF LIST, Y IS A LIST ###

In [101]:
#Split the data into 80% training and 20% testing
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [102]:
# Create and train the model (Support Vector Machine [Regressor])
svr_rbf= SVR(kernel = 'rbf', C= 1e3, gamma= 0.1)
svr_rbf.fit(x_train, y_train)


SVR(C=1000.0, gamma=0.1)

In [103]:
#testing the model: Score returns the coefficient of determination R^2 of the prediction
#The best possible score is 1.0
svm_confidence= svr_rbf.score(x_test, y_test)
print('svm_confidence: ', svm_confidence)


svm_confidence:  0.991400967938369


In [104]:
#Create and train the linear regression model
lr= LinearRegression()
#train the model
lr.fit(x_train, y_train)

LinearRegression()

In [105]:
#testing the model: Score returns the coefficient of determination R^2 of the prediction
#The best possible score is 1.0
lr_confidence= lr.score(x_test, y_test)
print('lr_confidence: ', lr_confidence)

lr_confidence:  0.9911283134752211


In [106]:
#Set a variable 'x_forecast' equal to the last 'n' rows of the original dataset from 'Adj. Close' column
x_forecast = np.array(X)[-forecast_out:]
print(x_forecast)

[[169.23]
 [172.26]
 [172.23]
 [173.03]
 [175.  ]
 [174.35]
 [174.33]
 [174.29]
 [175.28]
 [177.09]
 [176.19]
 [179.1 ]
 [179.26]
 [178.46]
 [177.  ]
 [177.04]
 [174.22]
 [171.11]
 [171.51]
 [167.96]
 [166.97]
 [167.43]
 [167.78]
 [160.37]
 [157.49]
 [163.03]
 [159.54]
 [155.32]
 [155.97]
 [162.71]]


In [107]:
#Print the predictions for the next 'n' days using linear regression model
lr_predictions = lr.predict(x_forecast)
print('LR predictions')
print(lr_predictions)
print(' ')
#Print the predictions for the next 'n' days using support vector regressor model
svm_predictions = svr_rbf.predict(x_forecast)
print('SVM predictions')
print(svm_predictions)
print(' ')
print('lr_confidence: ', lr_confidence)
print('svm_confidence: ', svm_confidence)

LR predictions
[172.6814916  175.77089051 175.7403024  176.55598528 178.56460437
 177.90186203 177.88146996 177.84068581 178.85009338 180.69557589
 179.77793265 182.74497912 182.9081157  182.09243282 180.60381157
 180.64459571 177.76931356 174.59834637 175.00618781 171.38659503
 170.37718747 170.84620513 171.20306639 163.64780372 160.71134536
 166.35994929 162.80153273 158.49880555 159.16154789 166.03367614]
 
SVM predictions
[176.0303882  171.7086606  171.73391559 170.66529449 174.17708741
 171.45127949 171.38597234 171.26008381 175.45313    177.49191816
 178.0198731  176.45479718 176.10982441 176.71966925 177.61741298
 177.56207576 171.05549742 171.69331525 171.83453303 178.48506832
 174.16163497 176.83573572 178.11979976 155.11482269 171.05735301
 156.63101145 156.86017932 162.58445339 167.76408962 156.68426812]
 
lr_confidence:  0.9911283134752211
svm_confidence:  0.991400967938369
